# Fault Injection on Quantized Neural Networks

This notebook tests fault injection on quantized neural networks (QNNs). It is recommended to first read through and understand some of the example notebooks that demonstrate image classification with BNN-PYNQ.

## 1. Import the packages

In [1]:
import bnn
import os

## 2. Load the required datasets

This notebook utilizes the CIFAR-10, GTSRB, SVHN, and MNIST datasets. You can download them from each given url via wget and unzip it to a folder on Pynq as shown below.

#### CIFAR-10

In [2]:
if not os.path.exists("/home/xilinx/jupyter_notebooks/bnn/cifar-10-binary.tar.gz"):
    !wget https://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz

if not os.path.exists("/home/xilinx/jupyter_notebooks/bnn/cifar-10-batches-bin/"):
    !tar -xf cifar-10-binary.tar.gz

cifar_files, cifar_labels = bnn.util.load_cifar10_testset("/home/xilinx/jupyter_notebooks/bnn/cifar-10-batches-bin/", 1000)

#### GTSRB

In [3]:
if not os.path.exists("/home/xilinx/jupyter_notebooks/bnn/GTSRB_Final_Test_Images.zip"):
    !wget https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Test_Images.zip

if not os.path.exists("/home/xilinx/jupyter_notebooks/bnn/GTSRB_Final_Test_GT.zip"):
    !wget https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Test_GT.zip

if not os.path.exists("/home/xilinx/jupyter_notebooks/bnn/GTSRB"):
    !unzip -q -o GTSRB_Final_Test_Images.zip
    !unzip -q -o GTSRB_Final_Test_GT.zip
    !mv GT-final_test.csv GTSRB/Final_Test/

gtsrb_files, gtsrb_labels = bnn.util.load_gtsrb_testset(
    "/home/xilinx/jupyter_notebooks/bnn/GTSRB/Final_Test/GT-final_test.csv",
    "/home/xilinx/jupyter_notebooks/bnn/GTSRB/Final_Test/Images",
    1000
)

#### SVHN

In [4]:
if not os.path.exists("/home/xilinx/jupyter_notebooks/bnn/test_32x32.mat"):
    !wget http://ufldl.stanford.edu/housenumbers/test_32x32.mat

svhn_files, svhn_labels = bnn.util.load_svhn_testset("/home/xilinx/jupyter_notebooks/bnn/test_32x32.mat", 1000)

#### MNIST

In [5]:
if not os.path.exists("/home/xilinx/jupyter_notebooks/bnn/t10k-images-idx3-ubyte.gz"):
    !wget http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz 
    !wget http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz 
        
if not os.path.exists("/home/xilinx/jupyter_notebooks/bnn/t10k-images-idx3-ubyte"):
    !gzip -d t10k-images-idx3-ubyte.gz
    !gzip -d t10k-labels-idx1-ubyte.gz

mnist_files, mnist_labels = bnn.util.load_mnist_testset("/home/xilinx/jupyter_notebooks/bnn/")

--2020-03-03 03:31:03--  http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
Resolving yann.lecun.com (yann.lecun.com)... failed: Temporary failure in name resolution.
wget: unable to resolve host address ‘yann.lecun.com’
--2020-03-03 03:31:03--  http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz
Resolving yann.lecun.com (yann.lecun.com)... failed: Temporary failure in name resolution.
wget: unable to resolve host address ‘yann.lecun.com’


&nbsp;
## 3. A Simple Fault Test Example
This example demonstrates the most basic method of running a fault test, and demonstrates the use of all of the classification function arguments. The classes beyond this section expand on this concept and allow the user to more easily set up customizeable and comprehensive tests.

The test is performed in a way very simlar to how a normal image classification is performed. The main difference is that `classifier.classify_cifars_with_faults()` is called instead of `classifier.classify_cifars()`. This function takes additional arguments that specify the type, location, and number of faults.

For the CnvClassifier, non-CIFAR10 datasets (e.g. GTSRB and SVHN) will need to be converted to CIFAR-10 format before being classified. This will be automatically handled by using the `classifier.classify_images()` function instead of `classifier.classify_cifars()`. This function takes a list of Pillow Images and converts them to the required format before classifying. For fault injection with these datasets, use `classifier.classify_images_with_faults()`.

The inference can be performed with different precision for weights and activation. Creating a specific Classifier will automatically download the correct bitstream onto PL and load the weights and thresholds trained on the specific dataset.

In [6]:
print("Availabe params:", bnn.available_params(bnn.NETWORK_CNVW1A1))

# Instantiate the cnvW1A1 hardware classifier with the CIFAR10 dataset
classifier = bnn.CnvClassifier(bnn.NETWORK_CNVW1A1, 'cifar10', bnn.RUNTIME_HW)

# Classify the dataset, and inject 10000 MBUs, targeting both weights and thresholds in the first 3 layers.
#
# The default value of target_layers is an empty list, which will target all layers.
#
# target_type determines if weights or thresholds will be targeted. A value of -1 causes both to be targeted, a value
# of 0 causes weights to be targeted, and a value of 1 will target thresholds.
print("Classifying", len(cifar_labels), "images with", classifier.net + "-" + classifier.params)
results = classifier.classify_cifars_with_faults(cifar_files, num_faults=10000, flip_word=True, target_type=-1, target_layers=[0, 1, 2])

# Calculate the accuracy of the network
accuracy = bnn.util.calculate_accuracy(results, cifar_labels)

print("Accuracy:", accuracy)

Availabe params: ['cifar10', 'road-signs', 'streetview']
Classifying 1000 images with cnvW1A1-cifar10


/usr/local/lib/python3.6/dist-packages/bnn/bnn.py:179: UserWarning: implicit cast from 'char *' to a different pointer type: will be forbidden in the future (check that the types are as you expect; use an explicit ffi.cast() if they are correct)
  path.encode(), len(self.classes), size_ptr, usecperimage, num_faults, 1 if flip_word else 0, target_type, targets, len(target_layers)


Inference took 1582000.00 microseconds, 1582.00 usec per image
Classification rate: 632.11 images per second
Accuracy: 59.1


&nbsp;
## 4. Using the FaultTest Class

This example demonstrates the use of the FaultTest class. This class facilitates simpler automated testing of fault injections. Two examples are provided, the latter of which shows the use of all arguments. See `bnn/faults/faults.py` for a detailed description of the FaultTest class.

In [ ]:
from bnn.faults import FaultTest, CNVFaultTest

# Create an instance of the CNVFaultTest class that uses the cnvW1A1 network with the CIFAR-10 testset
cnv_test = CNVFaultTest.CIFARTest(bnn.NETWORK_CNVW1A1, cifar_files, cifar_labels)

# Test the network, accepting the default values for flip_word, target_type, and target_layers
results, times, accuracies = cnv_test.run_test(num_runs=5, num_flips=10000)
print('10000 Bit Flips (10 Tests): Accuracy =', sum(accuracies) / len(accuracies))
print('\n')

# Test the network, specifying all parameters
results, times, accuracies = cnv_test.run_test(
    num_runs = 5,
    num_flips = 10000,
    flip_word = True,
    target_type = FaultTest.TargetType.thresholds(),
    target_layers = [0, 1]
)
print('10000 Threshold Word Flips in Layers [0,1] (10 Tests): Accuracy =', sum(accuracies) / len(accuracies))

&nbsp;
## 5. Using the NetworkTest Class

This example demonstrates the use of the NetworkTest class. This class allows the user to easily execute a comprehensive test of a network. See `bnn/faults/faults.py` for a detailed description of the NetworkTest class.

In [ ]:
from bnn.faults import CNVFaultTest, NetworkTest

output_folder = "/home/xilinx/jupyter_notebooks/bnn/faults/"
cnv_test = CNVFaultTest.CIFARTest(bnn.NETWORK_CNVW1A1, cifar_files, cifar_labels)

# Use the FaultTest defined above to run all tests, and store results in the output folder defined above
net_test = NetworkTest(cnv_test, output_folder)


# Run the specified number of tests in the target layers. Separate tests will be
# executed using all combinations of the following fault counts and test types.
net_test.test_network(
    num_runs = 10,
    flip_counts = [10, 100, 1000],
    test_types = [NetworkTest.TestType.any_word(), NetworkTest.TestType.threshold_bit()],
    target_layers = [0, 1]
)

# Run a comprehensive test. Tests will be run for every test type.
net_test.comprehensive_test(
    num_runs = 10,
    flip_counts = [10, 100, 1000],
    target_layers = [0, 1, 2]
)

&nbsp;

## 6. Running Tests on All CNV Networks
The dictionary below instantiates tests for all of the available CNV networks. These tests are then conducted for each individual layer multiple times, with an increasing number of faults each time.

In [11]:
output_folder = "/home/xilinx/jupyter_notebooks/bnn/faults/"

num_runs = 100
flip_counts = [5, 10, 50, 100]

cnv_tests = {
    # cnvW1A1 Networks
    'cnvW1A1': [
        NetworkTest(CNVFaultTest.CIFARTest(bnn.NETWORK_CNVW1A1, cifar_files, cifar_labels), output_folder),
        NetworkTest(CNVFaultTest.SVHNTest(bnn.NETWORK_CNVW1A1, svhn_files, svhn_labels), output_folder),
        NetworkTest(CNVFaultTest.GTSRBTest(bnn.NETWORK_CNVW1A1, gtsrb_files, gtsrb_labels), output_folder),
    ],
    'cnvW1A1-TMR': [
        NetworkTest(CNVFaultTest.CIFARTest(bnn.NETWORK_CNVW1A1_TMR, cifar_files, cifar_labels), output_folder),
        NetworkTest(CNVFaultTest.SVHNTest(bnn.NETWORK_CNVW1A1_TMR, svhn_files, svhn_labels), output_folder),
        NetworkTest(CNVFaultTest.GTSRBTest(bnn.NETWORK_CNVW1A1_TMR, gtsrb_files, gtsrb_labels), output_folder),
    ],
    'cnvW1A1-interleaved': [
        NetworkTest(CNVFaultTest.CIFARTest(bnn.NETWORK_CNVW1A1_INTERLEAVED, cifar_files, cifar_labels), output_folder),
        NetworkTest(CNVFaultTest.SVHNTest(bnn.NETWORK_CNVW1A1_INTERLEAVED, svhn_files, svhn_labels), output_folder),
        NetworkTest(CNVFaultTest.GTSRBTest(bnn.NETWORK_CNVW1A1_INTERLEAVED, gtsrb_files, gtsrb_labels), output_folder),
    ],

    # cnvW1A2 Networks
    'cnvW1A2': [
        NetworkTest(CNVFaultTest.CIFARTest(bnn.NETWORK_CNVW1A2, cifar_files, cifar_labels), output_folder)
    ],
#    'cnvW1A2-TMR': [
#        NetworkTest(CNVFaultTest.CIFARTest(bnn.NETWORK_CNVW1A2_TMR, cifar_files, cifar_labels), output_folder)
#    ],
    'cnvW1A2-interleaved': [
        NetworkTest(CNVFaultTest.CIFARTest(bnn.NETWORK_CNVW1A2_INTERLEAVED, cifar_files, cifar_labels), output_folder)
    ],

    # cnvW2A2 Networks
    'cnvW2A2': [
        NetworkTest(CNVFaultTest.CIFARTest(bnn.NETWORK_CNVW2A2, cifar_files, cifar_labels), output_folder)
    ],
    'cnvW2A2-TMR': [
        NetworkTest(CNVFaultTest.CIFARTest(bnn.NETWORK_CNVW2A2_TMR, cifar_files, cifar_labels), output_folder)
    ],
    'cnvW2A2-interleaved': [
        NetworkTest(CNVFaultTest.CIFARTest(bnn.NETWORK_CNVW2A2_INTERLEAVED, cifar_files, cifar_labels), output_folder)
    ],
}

In [ ]:
weight_and_thresh = [TestType.weight_bit(), TestType.weight_word(), TestType.threshold_bit(), TestType.threshold_word()]
only_weights = [TestType.weight_bit(), TestType.weight_word()]

all_tests_flat = [test for lst in cnv_tests.values() for test in lst]


# CNV Network uses passthrough activations for the final layer (layer 8), so that one will be handled differently.
for test in all_tests_flat:
    test.test_network(num_runs, flip_counts, test_types=weight_and_thresh, target_layers=[0, 1, 2, 3, 4, 5, 6, 7])

for test in all_tests_flat:
    test.test_network(num_runs, flip_counts, test_types=only_weights, target_layers=[8])